In [1]:
import json
import csv
from collections import defaultdict

In [2]:
# Chargement des données
with open("alignement_mots_manuel_correct.json", "r", encoding="utf-8") as f:
    donnees = json.load(f)

In [3]:
# Structure : erreurs_groupées[correct][manuel][pays] = set(apprenants)
erreurs_groupées = defaultdict(lambda: defaultdict(lambda: defaultdict(set)))

# Traitement des données
for apprenant in donnees:
    pays = apprenant.get('pays', 'Inconnu')
    id_apprenant = apprenant.get('apprenant', 'ID_inconnu')

    erreurs_vues = set()  # pour ne pas doubler les erreurs par élève

    for segment in apprenant['segments']:
        for mot in segment['segments']:
            manuel = mot.get('manuel', '').strip()
            correct = mot.get('correct', '').strip()

            if manuel in {'-', '—'} or correct in {'-', '—'}:
                continue
            if manuel and correct and manuel != correct:
                cle_erreur = (correct, manuel)
                if cle_erreur not in erreurs_vues:
                    erreurs_groupées[correct][manuel][pays].add(id_apprenant)
                    erreurs_vues.add(cle_erreur)

In [4]:
# Sauvegarde dans un fichier CSV
with open("statistiques_erreurs_par_apprenant.csv", "w", newline="", encoding="utf-8") as fichier_csv:
    writer = csv.writer(fichier_csv)
    writer.writerow(["Forme correcte", "Forme erronée (manuel)", "Nombre d'apprenants", "FR", "IT", "NL", "UK", "GE", "Apprenants"])

    for correct, formes_fautives in erreurs_groupées.items():
        for manuel, pays_dict in formes_fautives.items():
            total_apprenants = sum(len(apprenants) for apprenants in pays_dict.values())
            
            # Zbierz wszystkie numery uczniów, którzy popełnili ten błąd, bez powtórzeń
            tous_les_apprenants = set()
            for apprenants in pays_dict.values():
                tous_les_apprenants.update(apprenants)
            
            # Posortuj i połącz w string oddzielony przecinkami
            liste_apprenants = ", ".join(sorted(tous_les_apprenants))
            
            ligne = [
                correct,
                manuel,
                total_apprenants,
                len(pays_dict.get('FR', set())),
                len(pays_dict.get('IT', set())),
                len(pays_dict.get('NL', set())),
                len(pays_dict.get('UK', set())),
                len(pays_dict.get('GE', set())),
                liste_apprenants
            ]
            writer.writerow(ligne)

print("Le fichier 'statistiques_erreurs_par_apprenant.csv' a été créé avec succès !")


Le fichier 'statistiques_erreurs_par_apprenant.csv' a été créé avec succès !


In [5]:
# Statistiques globales des erreurs
total_alignements = 0
total_erreurs = 0

for apprenant in donnees:
    for segment in apprenant["segments"]:
        for paire in segment["segments"]:
            manuel = paire.get('manuel', '').strip()
            correct = paire.get('correct', '').strip()

            if manuel in {'-', '—'} or correct in {'-', '—'}:
                continue

            total_alignements += 1
            if manuel != correct:
                total_erreurs += 1

pourcentage_erreurs = (total_erreurs / total_alignements * 100) if total_alignements > 0 else 0

print(f"\nNombre total de paires alignées : {total_alignements}")
print(f"Nombre de cas d’erreurs (manuel ≠ correct) : {total_erreurs}")
print(f"Pourcentage d’erreurs : {pourcentage_erreurs:.2f} %")



Nombre total de paires alignées : 3466
Nombre de cas d’erreurs (manuel ≠ correct) : 1352
Pourcentage d’erreurs : 39.01 %
